### Sprint 2: Model Development

In [1]:
# Feature Engineering for Water Pump Classification
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data_path = os.path.join(os.getcwd(), '..', 'data')
out_path = os.path.join(os.getcwd(), '..', 'outputs')     # For the output

# Read the files
train = pd.read_csv(os.path.join(data_path, 'cleaned_data_filled_V6.csv'))
labels = pd.read_csv(os.path.join(data_path, 'train_labels.csv'))

# Merge training labels
df = pd.merge(train, labels, on='id')
print('Files successfully loaded and merged!')


Files successfully loaded and merged!


## Ticket 2.1.1: New Features 

In [6]:
# Age of pump
current_year = 2025
df['pump_age'] = current_year - df['construction_year']

# Manual vs. motorized extraction
df['is_manual_pump'] = df['extraction_type_class'].str.lower().str.contains('hand').astype(int)

# water_risk_score

# 1. Water Quality + Quantity Combination: Create 'water_risk_score'
# Map water_quality into numeric risk levels
water_quality_map = {
    'soft': 1, 'good': 1,
    'salty': 2, 'milky': 2,
    'coloured': 3, 'fluoride': 3, 'fluoride, salty': 4, 'salty abandoned': 4
}
df['water_quality_numeric'] = df['water_quality'].map(water_quality_map)

# Combine amount_tsh_capped and water quality
df['water_risk_score'] = df['amount_tsh_capped'] * df['water_quality_numeric']

## Ticket 2.1.2: Transform existing features to improve model performance 